In [ ]:
!pip install sidetable -q

In [ ]:
import numpy as np
import pandas as pd
import sidetable

import statsmodels.api as sm


from statsmodels.tsa.stattools import adfuller
import holidays
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import plotly.express as px


from sklearn.preprocessing import (
    MinMaxScaler
)


from sklearn.model_selection import (
    KFold,StratifiedKFold,cross_val_score, train_test_split
)

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)

import xgboost as xgb

import plotly.graph_objects as go
from scipy import stats

# Importing pipelines
from sklearn.pipeline import  Pipeline
from sklearn.compose import(
    ColumnTransformer,
)

from sklearn.base import (
    BaseEstimator, TransformerMixin
)

# Importing class, to load and save predictive models in external files.
import pickle,joblib



import warnings
warnings.filterwarnings('ignore')

In [ ]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(labels=self.columns, axis=1)

In [ ]:
def decomposition(df, date_col, target_col, filial_col):
    df = df.copy()

    # Garantir datetime e índice
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col)

    # Criar colunas de período
    df['ano']    = df.index.to_period('Y')
    df['mes']    = df.index.to_period('M')
    df['semana'] = df.index.to_period('W')
    df['dia']    = df.index.to_period('D')

    # Totais por filial + período
    df['qtd_per_year']  = df.groupby([filial_col, 'ano'])[target_col].transform('sum')
    df['qtd_per_month'] = df.groupby([filial_col, 'mes'])[target_col].transform('sum')
    df['qtd_per_week']  = df.groupby([filial_col, 'semana'])[target_col].transform('sum')
    df['qtd_per_day']   = df.groupby([filial_col, 'dia'])[target_col].transform('sum')

    # Médias por filial + período
    df['avg_qtd_per_year']  = df.groupby([filial_col, 'ano'])[target_col].transform('mean')
    df['avg_qtd_per_month'] = df.groupby([filial_col, 'mes'])[target_col].transform('mean')
    df['avg_qtd_per_week']  = df.groupby([filial_col, 'semana'])[target_col].transform('mean')
    df['avg_qtd_per_day']   = df.groupby([filial_col, 'dia'])[target_col].transform('mean')

    # Restaurar índice original
    return df.reset_index()


In [ ]:
def list_holidays(df):

    df['date']= pd.to_datetime(df.date)

    min_year = df.date.min().year
    max_year = df.date.max().year

    years_list = pd.period_range(min_year, max_year, freq='Y')

    list_of_holidays = []
    for year in years_list:
        list_of_holidays.append(holidays.BR(years=int(str(year))).keys())

    holiday_list = [item for sublist in list_of_holidays for item in sublist]

    return holiday_list

In [ ]:
def create_dt_attributes(df):
    df['date'] = pd.to_datetime(df.date)
    df['month'] = df.date.dt.month
    df['day_of_month'] = df.date.dt.day
    df['day_of_year'] = df.date.dt.dayofyear
    df['week_of_year'] = df.date.dt.isocalendar().week
    df['day_of_week'] = df.date.dt.weekday + 1
    df['year'] = df.date.dt.year
    df['is_weekend'] = df.date.dt.weekday // 5
    df['start_of_month'] = df['day_of_month'].apply(lambda x: 1 if x <= 5 else 0)
    df['end_of_month']   = df['day_of_month'].apply(lambda x: 1 if x >= 25 else 0)
    df['is_holiday'] = np.where(df.date.isin(list_holidays(df)), 1, 0)

    return df

In [ ]:
def test_stationarity(serie):
    # Calcula estatísticas móveis
    rolmean = serie.rolling(window=12).mean()
    rolstd = serie.rolling(window=12).std()

    # Plot das estatísticas móveis
    orig = plt.plot(serie, color='blue', label='Original')
    mean = plt.plot(rolmean, color='red', label='Média Móvel')
    std = plt.plot(rolstd, color='black', label='Desvio Padrão')

    # Plot
    plt.legend(loc='best')
    plt.title('Estatísticas Móveis - Média e Desvio Padrão')
    plt.show()

    # Teste Dickey-Fuller:
    # Print
    print('\nResultado do Teste Dickey-Fuller:\n')

    # Test
    dfteste = adfuller(serie, autolag='AIC')

    # Formatting the output
    dfsaida = pd.Series(dfteste[0:4], index=['Estatística do Teste',
                                             'Valor-p',
                                             'Número de Lags Consideradas',
                                             'Número de Observações Usadas'])

    # Loop por cada item da saída do teste
    for key, value in dfteste[4].items():
        dfsaida['Valor Crítico (%s)' % key] = value

    # Print
    print(dfsaida)

    # Testa o valor-p
    print('\nConclusão:')
    if dfsaida[1] > 0.05:
        print('\nO valor-p é maior que 0.05 e, portanto, não temos evidências para rejeitar a hipótese nula.')
        print('Essa série provavelmente não é estacionária.')
    else:
        print('\nO valor-p é menor que 0.05 e, portanto, temos evidências para rejeitar a hipótese nula.')
        print('Essa série provavelmente é estacionária.')

In [ ]:
# Creating a function to select the best features
def feature_imp(features, target,param_imp,n_best_features):
    # Define o classificador, ou seja, instância um objeto da classe XGBRegressor
    reg_XBGR = xgb.XGBRFRegressor(verbosity=0, silent=True)

    # ajuste os dados
    reg_XBGR.fit(features, target)

    # selecionando os melhores parâmetros com grid search, que indicar a importância relativa de cada atributo para fazer previsões precisas:
    reg_XBGR_feature_imp = reg_XBGR.get_booster().get_score(importance_type=param_imp)

    # obtém nome das colunas
    keys = list(reg_XBGR_feature_imp.keys())

    # obtém scores das features
    values = list(reg_XBGR_feature_imp.values())

    # crianndo dataframe  com  k recusros principais
    xbg_best_features = pd.DataFrame(data=values, index=keys, columns=["score_XGBRFRegressor"]).sort_values(
        by="score_XGBRFRegressor", ascending=True).nlargest(n_best_features, columns="score_XGBRFRegressor")

    # Return the best features
    return xbg_best_features

In [ ]:
def xgb_model_helper(X_train, y_train, PARAMETERS, V_PARAM_NAME=False, V_PARAM_VALUES=False, BR=10,):
    # Cria uma matrix temporária em formato de bit do conjunto de dados a ser treinados
    temp_dmatrix = xgb.DMatrix(data=X_train, label=y_train)

    # Check os parâmetros a ser utilizados
    if V_PARAM_VALUES == False:
        cv_results = xgb.cv(dtrain=temp_dmatrix, nfold=5, num_boost_round=BR, params=PARAMETERS, as_pandas=True,
                            seed=123)
        return cv_results

    else:
        # Criando uma Lista, para armazenar os resultados e os nomes, de cada uma das métricas.
        results = []

        # Percorre a lista de parâmetros
        for v_param_value in V_PARAM_VALUES:
            # Adicionando o nome dos parâmetros avaliado a lista de nomes.
            PARAMETERS[V_PARAM_NAME] = v_param_value

            # Treinando o modelo com Cross Validation.
            cv_results = xgb.cv(dtrain=temp_dmatrix, nfold=5, num_boost_round=BR, params=PARAMETERS, as_pandas=True,
                                seed=123)

            # Adicionando os resultados gerados a lista de resultados.
            results.append((cv_results["train-mae-mean"].tail().values[-1], cv_results["test-mae-mean"].tail().values[
                -1]))  # .tail().values[-1] captura somente as colunas

        # zip “pareia” os elementos de uma série de listas, tuplas ou outras sequências para criar uma lista de tuplas:

        # Adicionando a média da AUC e o desvio-padrão dos resultados gerados, pelo modelo analisado ao Dataframe de médias.
        data = list(zip(V_PARAM_VALUES, results))
        print(pd.DataFrame(data, columns=[V_PARAM_NAME, "mae"]))

        return cv_results

In [ ]:
def opt_number_of_boosting_rounds(X_train, y_train,):
    # create the DMatrix
    temp_dmatrix = xgb.DMatrix(data=X_train, label=y_train)

    # Create the parameter dictionary for each tree: params
    params = {"objective": 'reg:linear', "max_depth": 5}

    # Create lis of number of boosting rounds
    num_rounds = [5, 10, 20, 25, 50, 100]

    # Empty list to store final round rmse per XGBoost model
    final_rmse_per_round = []

    # Iterate ove num_rounds and build one model per num_boost_round parameter
    for curr_num_rounds in num_rounds:
        # Perform cross-validation: cv_results
        cv_results = xgb.cv(dtrain=temp_dmatrix, params=params, nfold=5, num_boost_round=curr_num_rounds,
                            metrics="mae", as_pandas=True, seed=123)
        # Append final round RMSE
        final_rmse_per_round.append(cv_results["test-mae-mean"].tail().values[-1])
    # print the resultant Dataframe
    num_rounds_rmses = list(zip(num_rounds, final_rmse_per_round))

    return pd.DataFrame(num_rounds_rmses, columns=["num_boosting_rounds", "mae"])

In [ ]:
def regression(model,x_test,y_test):

    # Evaluating the model
    y_pred  = model.predict(x_test)
    print("Metrics in Test data:\n ")
    print('='*30)
    # Calculate the error
    print('MAE:', mean_absolute_error(y_test,y_pred))
    print('MAPE:',mean_absolute_percentage_error(y_test,y_pred))
    print('MSE:', mean_squared_error(y_test,y_pred))
    #print('RMSE:',mean_squared_error(y_test,y_pred, squared = False))
    print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
    print('R2:',  r2_score(y_test, y_pred))

In [ ]:
def plot_preditction(y_test,y_pred,title,template):
    HEIGHT = 700
    WIDTH = 950
    TITLE_FONT={'size':20, 'family': 'Times New Roman',}
    TITLE_X=0.5
    FONT_COLOR = "#000000"
    # Create the figures
    fig = go.Figure([
        #go.Scatter(y=y_train, name='train', mode='markers'),
        go.Scatter(y=y_test, name='test', mode='markers'),
        go.Scatter(y=y_pred, name='prediction')
    ])

    fig.update_layout(title_text=title,
                      title_x=TITLE_X,
                      title_font=TITLE_FONT,
                      font_color=FONT_COLOR,
                      height= HEIGHT,
                      width= WIDTH,
                      xaxis_title='Qtd. per Week',
                      yaxis_title='Forecast',
                      template= template,
                     )
    fig.show()

In [ ]:
df = pd.read_excel('/content/dataset_25.xlsx')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
df = df.groupby(['date','filial'])['vol'].sum().reset_index()

In [ ]:
# Create copy dataframe
df_cp = df.copy()

# Remove todas as colunas indesejadas
unwanted_columns = ['familia','class_prod','categ_prod','tipo_grupo', 'tipo_mfm', 'código tipo manfrim', '1']
df.drop(unwanted_columns, axis = 1, inplace = True, errors = 'ignore')

In [ ]:
# Combination of attributes
df['vol'] = df['vol'] / 15
# ou, se quiser renomear a coluna depois
df.rename(columns={'vol': 'qtd_vendida'}, inplace=True)

# Ordem cronológica da série
df = df.sort_values('date').reset_index(drop=True)

# View combination
df.head()

In [ ]:
df.info()

In [ ]:
# Identificando se existe Datas Faltantes

# Gerando todas as datas do intervalo esperado
datas_completas = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')

# Verificando quais datas estão faltando
datas_faltando  = datas_completas.difference(df['date'])

datas_faltando

In [ ]:
chaves = ['filial']

# Datas completas
df_datas = pd.DataFrame({'date': pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')})

# Combinações únicas de chaves
chaves_unicas = df[chaves].drop_duplicates()

# Produto cartesiano
df_base = chaves_unicas.merge(df_datas, how='cross')

# Merge com dados reais
df = df_base.merge(df[['date'] + chaves + ['qtd_vendida']], on=['date'] + chaves, how='left')

# Preenchimento
df['qtd_vendida'] = df['qtd_vendida'].fillna(0)

In [ ]:
df.info()

In [ ]:
#df_agg = (
#    df
#    .groupby([
#        pd.Grouper(key='date', freq='W'),  # agrupa por semana
#        'filial'
#    ])['qtd_vendida']
#    .sum()
#    .reset_index()
#    .sort_values(['filial', 'date'])
#)

In [ ]:
# Function call, time series decomposition
# ts_decomp= decomposition(df_agg,'date','filial','qtd_vendida')
ts_decomp = decomposition(df, date_col='date', target_col='qtd_vendida', filial_col='filial')


# View decomposition
ts_decomp.head()

In [ ]:
# Checking for duplicate records.
print(f'Values duplicated: {ts_decomp.duplicated().sum()}')

In [ ]:
ts_decomp.info()

In [ ]:
df_dropna = ts_decomp.copy()

In [ ]:
# Drop duplicates
df_drop_dup=ts_decomp.drop_duplicates(subset=['date'],keep='last')


In [ ]:
df_drop_dup.stb.missing(clip_0=True, style=True)


In [ ]:
# Drop on column axis with nan values
df_dropna=df_drop_dup.dropna(subset=['qtd_per_week'])

# Analyzing if there are missing values in the dataset
df_dropna.stb.missing(style=True)

In [ ]:
# Remove as colunas indesejadas
unwanted_columns = ['avg_qtd_per_year','qtd_per_year','qtd_per_month','avg_qtd_per_month', 'semana', 'ano','mes','dia', 'qtd_vendida']

df_dropna.drop(unwanted_columns, axis = 1, inplace = True, errors = 'ignore')

# Checking for duplicate records.
print(f'Values duplicated: {df_dropna.duplicated().sum()}')

In [ ]:
df_dropna.info()

In [ ]:
 df_dropna.groupby('filial').tail(3)


In [ ]:
def preparar_lags_semanais(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])

    # Cria colunas auxiliares apenas para o agrupamento
    df['ano'] = df['date'].dt.isocalendar().year
    df['semana'] = df['date'].dt.isocalendar().week
    df['inicio_semana'] = df['date'] - pd.to_timedelta(df['date'].dt.weekday, unit='D')

    # Agrega por semana e filial
    semanal = (
        df.groupby(['filial', 'ano', 'semana'], as_index=False)
          .agg({
              'qtd_per_day': 'sum',
              'inicio_semana': 'first'
          })
          .rename(columns={'qtd_per_day': 'qtd_per_week', 'inicio_semana': 'date'})
    )

    # Ordena corretamente por semana
    semanal = semanal.sort_values(['filial', 'date'])

    # Calcula lags e médias móveis defasadas
    semanal['lag_1'] = semanal.groupby('filial')['qtd_per_week'].shift(1)
    semanal['lag_2'] = semanal.groupby('filial')['qtd_per_week'].shift(2)
    semanal['roll_mean_3'] = (
        semanal.groupby('filial')['qtd_per_week']
        .shift(1)
        .rolling(3)
        .mean()
        .reset_index(drop=True)
    )
    semanal['roll_mean_4'] = (
        semanal.groupby('filial')['qtd_per_week']
        .shift(1)
        .rolling(4)
        .mean()
        .reset_index(drop=True)
    )

    # Remove colunas auxiliares (ano, semana)
    semanal = semanal.drop(columns=['ano', 'semana'])

    return semanal





In [ ]:
df_new = preparar_lags_semanais(df_dropna)

In [ ]:
df_new.groupby('filial').tail(5)

In [ ]:
# Function SMA and MSD
def sma_and_msd(df):

    # Copy of dataframe
    df_copy = df.copy()

    # Cria a variável com o retorno (mudança percentual do total sales per week)
    # This will our variable target
    df_copy["perc_return"] = df_copy["qtd_per_week"].pct_change(1) * 100

    # Shift das colunas op sales toys
    df_copy["tsw"] = df_copy["qtd_per_week"].shift(1)

    # Calculando os OKR's

    # Simple Moving Average (SMA)
    df_copy["SMA_3"] = df_copy[["qtd_per_week"]].rolling(3).mean().shift(1)
    df_copy["SMA_5"] = df_copy[["qtd_per_week"]].rolling(5).mean().shift(1)

    # Moving Standard Deviation (MSD) - Volatilidade
    df_copy["MSD_3"] = df_copy["perc_return"].rolling(3).std().shift(1)
    df_copy["MSD_5"] = df_copy["perc_return"].rolling(5).std().shift(1)

    return df_copy.dropna()

# Function call
df_new_01 = sma_and_msd(df_new)


In [ ]:
df_ts = create_dt_attributes(df_new_01)
df_ts.head()

In [ ]:
df_ts.info()

In [ ]:
# Converte variáveis categóricas em variáveis dummy aplicando One-Hot Encoding
df_dummies = pd.get_dummies(df_ts, columns = [ 'month','year', 'filial'])


# Checking for duplicate records.
print(f'\nValues duplicated: {df_dummies.duplicated().sum()}')



In [ ]:
ts_regular = df_dummies[(df_dummies['date'] >='2025-05-31')].index
df_dummies.drop(ts_regular, inplace=True)


In [ ]:
df_dummies.info()

In [ ]:
df_total = df_dummies.groupby('date').agg({'qtd_per_week': 'sum'}).sort_index()
df_total = df_total.asfreq('W-MON')  # fixar frequência semanal
df_total['qtd_per_week'] = df_total['qtd_per_week'].fillna(0)

# Decomposição
addi = sm.tsa.seasonal_decompose(df_total['qtd_per_week'], model='additive', extrapolate_trend='freq', period=7)
mult = sm.tsa.seasonal_decompose(df_total['qtd_per_week'], model='multiplicative', extrapolate_trend='freq', period=7)

# Plot
plt.rcParams.update({'figure.figsize': (16,10)})
addi.plot().suptitle('Additive Decomposition – Total Filiais', fontsize=22)
mult.plot().suptitle('Multiplicative Decomposition – Total Filiais', fontsize=22)
plt.show()

# Teste ADF
test_stationarity(df_total['qtd_per_week'])


In [ ]:
# Colunas sem informações úteis ou com informações já derivadas serão descartadas.
cols = [col for col in df_dummies.columns if col not in ["date", "year", "month","day_of_week","qtd_per_week",]]



xFeatures, y = df_dummies[cols], df_dummies['qtd_per_week']


In [ ]:
xFeatures.info()

In [ ]:
xFeatures.head()

In [ ]:
# Function call Feature Importance
xbg_best_features = feature_imp(features=xFeatures,
                      target=y,
                      param_imp='weight',
                      n_best_features=7)

# Capturing k best features for model training
xbg_best_features

In [ ]:
extras = [
    'start_of_month',
    'end_of_month',
    'year_2023',
    'year_2024',
    'year_2025'
]
# Variáveis selecionadas automaticamente
auto_features = xbg_best_features.index.tolist()

# Junta e remove duplicatas
final_features = list(set(auto_features + extras))

final_features = ['year_2024',
 'start_of_month',
 'year_2025',
 'end_of_month',
 'lag_2',
 'day_of_year',
 'lag_1',
 'year_2023',
 'day_of_month',
 'roll_mean_3',
 'roll_mean_4',
 'filial_1',
 'filial_3',
 'filial_4',
 'filial_5',
 'filial_7',
 'roll_mean_3',
 'roll_mean_4',
  'month_1',
  'month_2',
  'month_3',
  'month_4',
  'month_5',
  'month_6',
  'month_7',
  'month_8',
  'month_9',
  'month_10',
  'month_11',
  'month_12'
]



In [ ]:
# separando variáveis em componentes de input e output

# semente do gerador de números aleatórios
SEED = 42

# Separaremos 20% dos dados para testes
TEST_SIZE= 0.2

# Creating new dataframe with top k features
#X = xFeatures.loc[:,xbg_best_features.index]
X = xFeatures[final_features]


# divisão dos dados train/test
X_train, X_test, y_train, y_test =train_test_split(X,y,random_state=SEED,test_size=TEST_SIZE)

# shape
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# Define Padronizador
scaled_train = MinMaxScaler()

# Aplicação da normalização utilizando a Function MinMaxScaler(), ou seja, transforme os dados
X_train_min_max = scaled_train.fit_transform(X_train)

# Aplicamos o fit somente nos dados de treino e aplicamos o transform nos dados de teste
X_test_min_max = scaled_train.transform(X_test)

# Padronizador da variável target
scaler_target = MinMaxScaler()

# Aplicação da normalização utilizando a Function MinMaxScaler(), ou seja, transforme os dados
y_train_min_max = scaler_target.fit_transform(y_train.to_numpy().reshape(-1,1))

# Aplicamos o fit somente nos dados de treino e aplicamos o transform nos dados de teste
y_test_min_max = scaler_target.transform(y_test.to_numpy().reshape(-1,1))

In [ ]:
# Specify the desired parameters
PARAMETERS={"objective":'reg:linear',
            "eval_metric":"mae",
            "booster":'gblinear',}

# Function call
xgb_model_helper(X_train_min_max,y_train,PARAMETERS)

In [ ]:
PARAMETERS={"objective":'reg:linear',
            "booster":'gblinear',
            "eval_metric":"mae",
            "learning_rate": 0.5,
}
xgb_model_helper(X_train_min_max,y_train,PARAMETERS)

In [ ]:
print(f'Qtd: {np.mean(y):,.2f}')

In [ ]:
opt_number_of_boosting_rounds(X_train_min_max,y_train)


In [ ]:
PARAMETERS={"objective":'reg:linear',
            "booster":'gblinear',
            "eval_metric":"mae",
            "learning_rate": 0.5, # Control Overfitting
}
V_PARAM_NAME="max_depth"
V_PARAM_VALUES=range(3,11,1)

data=xgb_model_helper(X_train_min_max,y_train,PARAMETERS,V_PARAM_NAME=V_PARAM_NAME,V_PARAM_VALUES=V_PARAM_VALUES,BR=25)

In [ ]:
PARAMETERS={"objective":'reg:linear',
            "booster":'gblinear',
            "eval_metric":"mae",
            "learning_rate": 0.5,
            "max_depth":4,

}
V_PARAM_NAME="gamma"
V_PARAM_VALUES = np.linspace(start=1e-2, stop=2, num=20).tolist()

data=xgb_model_helper(X_train_min_max,y_train,PARAMETERS,V_PARAM_NAME=V_PARAM_NAME,V_PARAM_VALUES=V_PARAM_VALUES,BR=25)

In [ ]:
PARAMETERS={"objective":'reg:linear',
            "booster":'gblinear',
            "eval_metric":"mae",
            "learning_rate": 0.5,
            "max_depth":4,
            "gamma":0.42,
}
V_PARAM_NAME="subsample"
V_PARAM_VALUES = [.5,.6,.7,.75,.8,.85,.9,]

data=xgb_model_helper(X_train_min_max,y_train,PARAMETERS,V_PARAM_NAME=V_PARAM_NAME,V_PARAM_VALUES=V_PARAM_VALUES,BR=25)

In [ ]:
PARAMETERS={"objective":'reg:linear',
            "booster":'gblinear',
            "eval_metric":"mae",
            "learning_rate": 0.5,
            "max_depth":4,
            "gamma":0.42,
            "subsample":0.90,
}
V_PARAM_NAME = "colsample_bytree"
V_PARAM_VALUES = [.5,.6,.7,.75,.8,.85,.9,]

data=xgb_model_helper(X_train_min_max,y_train,PARAMETERS,V_PARAM_NAME=V_PARAM_NAME,V_PARAM_VALUES=V_PARAM_VALUES,BR=25)

In [ ]:
PARAMETERS={"objective":'reg:linear',
            "booster":'gblinear',
            "eval_metric":"mae",
            "learning_rate": 0.5,
            "max_depth":4,
            "gamma":0.42,
            "subsample":0.9,
            "colsample_bytree":0.75
}
V_PARAM_NAME = "reg_alpha"
V_PARAM_VALUES = np.linspace(start=1e-3, stop=2, num=40).tolist()

data=xgb_model_helper(X_train_min_max,y_train,PARAMETERS,V_PARAM_NAME=V_PARAM_NAME,V_PARAM_VALUES=V_PARAM_VALUES,BR=25)

In [ ]:
PARAMETERS={"objective":'reg:linear',
            "booster":'gblinear',
            "eval_metric":"mae",
            "learning_rate": 0.5,
            "max_depth":4,
            "gamma":0.42,
            "subsample":0.90,
            "colsample_bytree":0.75,
            "reg_alpha":2
}
V_PARAM_NAME = "reg_lambda"
V_PARAM_VALUES = np.linspace(start=1e-3, stop=2, num=40).tolist()

data=xgb_model_helper(X_train_min_max,y_train,PARAMETERS,V_PARAM_NAME=V_PARAM_NAME,V_PARAM_VALUES=V_PARAM_VALUES,BR=25)

In [ ]:
PARAMETERS={"objective":'reg:linear',
            "booster":'gblinear',
            "eval_metric":"mae",
            "max_depth":4,
            "gamma":0.42,
            "subsample":0.90,
            "colsample_bytree":0.75,
            "reg_alpha":2,
            "reg_lambda": 1e-3
}
V_PARAM_NAME = "learning_rate"
V_PARAM_VALUES = np.linspace(start=1e-3, stop=0.5, num=40).tolist()

data=xgb_model_helper(X_train_min_max,y_train,PARAMETERS,V_PARAM_NAME=V_PARAM_NAME,V_PARAM_VALUES=V_PARAM_VALUES,BR=25)

In [ ]:
PARAMETERS={"objective":'reg:linear',
            "booster":'gblinear',
            "eval_metric":"mae",
            "max_depth":4,
            "gamma":0.42,
            "subsample":0.90,
            "colsample_bytree":0.75,
            "reg_alpha":2,
            "reg_lambda": 1e-3,
            "learning_rate":0.5
}
# Creating an instance of the XGBRegressor model class
reg_XGB = xgb.XGBRegressor(objective="reg:linear",
                           booster="gblinear",
                           eval_metric="mae",
                           max_depth=4,
                           subsample=0.9,
                           colsample_bytree=0.75,
                           reg_alpha=2,
                           reg_lambda=1e-3,
                           learning_rate=0.5,
                           gamma=0.42,
                           num_boost_round=25,
                           n_estimators=500,
)
# Fit the data (train) the model
reg_XGB.fit(X_train_min_max, y_train)

# Model Score, Evaluating Better Using Cross Validation
reg_xgb_rmse = -cross_val_score(reg_XGB, X_train_min_max, y_train, cv =10,scoring="neg_root_mean_squared_error")

# Printing the result
print('Summary')
print('='*20)
pd.options.display.float_format = "{:,.2f}".format
pd.Series(reg_xgb_rmse).describe()

In [ ]:
reg_XGB.set_params


In [ ]:
pd.options.display.float_format = "{:,.2f}".format
evaluate=regression(reg_XGB, X_test_min_max, y_test)

In [ ]:
# Create an the new dataframe with predictions
df_predict, df_predict['predictions'] = pd.DataFrame(y_test), reg_XGB.predict(X_test_min_max)
df_predict = df_predict.sort_index(ascending=True)

# View predict
df_predict

In [ ]:
df_predict['predictions'] = df_predict['predictions'].clip(lower=0)


In [ ]:
plot_preditction(y_test=df_predict.qtd_per_week,
                 y_pred=df_predict.predictions,
                 title='<b>Qtd. Forecast by Week</b>',
                 template='xgridoff'
                )

In [ ]:
# calculate confidence interval for the RMSE
def ci_rmse(y_pred, y_test, confidence = .95):

    # Capture quadratic errors
    squared_errors = (y_pred - y_test) ** 2

    # Return CI
    return np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),scale=stats.sem(squared_errors)))

# Function call
ci_rmse(reg_XGB.predict(X_test_min_max), y_test)

In [ ]:
# Coeficientes
df_coef= pd.DataFrame(reg_XGB.coef_, X.columns, columns = ['coefficient']).sort_values('coefficient',ascending= False)

# set colors
colors = ['Positive' if c > 0 else 'Negative' for c in df_coef.coefficient]

fig = px.bar(
    x=df_coef.index,y=df_coef.coefficient, color=colors,
    color_discrete_sequence=['green', 'red'],
    labels=dict(x='Features', y='Linear coefficient'),
    title='<b>Weight of each resource for forecasting total sales per week</b>'
)

fig.update_layout(title_font={'size':20, 'family': 'Times New Roman',},
                  font_color="#000000",
                  height=700,
                  template= "xgridoff",)
fig.show()

In [ ]:
challenge_columns = df_cp.columns

features = X.columns.tolist()
target = ['qtd_per_week']

# Remove todas as colunas indesejadas
unwanted_columns = list((set(challenge_columns) - set(target)) - set(features))
unwanted_columns


In [ ]:
# Criando uma instância do transformador, passando como parâmetro as colunas que não queremos
unwanted_columns = [col for col in unwanted_columns if col in X_train.columns]


drop_columns = DropColumns(unwanted_columns)


#Criando um pipeline completo com preparação e previsão
full_pipeline_with_predictor = Pipeline([
                        ('drop_cols',drop_columns),
                         ('scaler',MinMaxScaler()),
                         ('reg_XGB',reg_XGB)
])
full_pipeline_with_predictor

# Ajusta o pipeline aos dados de treino
full_pipeline_with_predictor.fit(X_train, y_train)






In [ ]:
joblib.dump(full_pipeline_with_predictor , "/content/model/ts_total_sales_per_week.pkl")


In [ ]:
# loading the model
model_loaded = joblib.load("/content/model/ts_total_sales_per_week.pkl")
model_loaded